In [16]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import linregress, pearsonr
from pyhdf.SD import SD, SDC
from datetime import datetime, timedelta


def process_aeronet_data(file_path, output_path):
    df = pd.read_csv(file_path, skiprows=6)
    df.columns = df.columns.str.strip()

    df['Time(hh:mm:ss)'] = df['Time(hh:mm:ss)'].str[:-3]
    df['dateTime'] = pd.to_datetime(df['Date(dd:mm:yyyy)'] + ' ' + df['Time(hh:mm:ss)'],
                                    format='mixed')

    df['dateTime'] = df['dateTime'].dt.strftime('%m/%d/%Y %I:%M:%S %p')

    df['AOD_550nm'] = df['AOD_500nm'] * (550 / 500) ** (-df['500-870_Angstrom_Exponent'])

    output_df = pd.DataFrame({
        'dateTime': df['dateTime'],
        'Latitude': df['Site_Latitude(Degrees)'],
        'Longitude': df['Site_Longitude(Degrees)'],
        'AOD_550nm': df['AOD_550nm']
    })

    output_df.to_csv(output_path, index=False)
    

# ============================
# MODIS Data Extraction
# ============================
def parse_datetime_from_filename(filename):
    parts = filename.split('.')
    year = int(parts[1][1:5])
    doy = int(parts[1][5:8])
    hour = int(parts[2][:2])
    minute = int(parts[2][2:4])
    return datetime(year, 1, 1) + timedelta(days=doy - 1, hours=hour, minutes=minute)

def extract_modis_data(file_list_path, hdf_dir, output_path):
    data_fields = [
        "AOD_550_Dark_Target_Deep_Blue_Combined",
        "AOD_550_Dark_Target_Deep_Blue_Combined_QA_Flag"
    ]
    data_list = []

    with open(file_list_path, 'r', encoding="utf-8") as f:
        files = f.readlines()

    for file_name in files:
        file_name = file_name.strip()
        full_path = os.path.join(hdf_dir, file_name)

        try:
            hdf = SD(full_path, SDC.READ)
            dt = parse_datetime_from_filename(file_name)
            lat = hdf.select("Latitude")[:].flatten()
            lon = hdf.select("Longitude")[:].flatten()

            record = {
                "datetime": [dt] * len(lat),
                "latitude": lat,
                "longitude": lon
            }

            for field in data_fields:
                sds = hdf.select(field)
                attrs = sds.attributes()
                data = sds.get().flatten().astype(np.float32)
                data = data * attrs.get("scale_factor", 1)
                data[data == attrs.get("_FillValue", -9999)] = np.nan
                record[field] = data

            temp_df = pd.DataFrame(record)
            temp_df = temp_df[temp_df["AOD_550_Dark_Target_Deep_Blue_Combined_QA_Flag"] >= 3]
            data_list.append(temp_df)

        except Exception as e:
            print(f"Error reading {full_path}: {e}")

    result_df = pd.concat(data_list, ignore_index=True)
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    result_df.to_csv(output_path, index=False)

# ============================
# Region Bound Calculation
# ============================
def calculate_bounds(file_path, box_size=0.15):
    df = pd.read_csv(file_path)
    lat = df['Latitude'].iloc[0]
    lon = df['Longitude'].iloc[0]
    half_box = box_size / 2
    return lat - half_box, lat + half_box, lon - half_box, lon + half_box

# ============================
# MODIS Spatial Filtering
# ============================
def spatial_filter_modis_data(df, bounds):
    lat_min, lat_max, lon_min, lon_max = bounds
    filtered_data = []

    for i in range(len(df)):
        lat = df["latitude"].iloc[i]
        lon = df["longitude"].iloc[i]
        aod = df["AOD_550_Dark_Target_Deep_Blue_Combined"].iloc[i]
        dt = df["datetime"].iloc[i]

        mask = (lat >= lat_min) & (lat <= lat_max) & (lon >= lon_min) & (lon <= lon_max)
        box_aod = aod[mask] 

        if len(box_aod) > 0:
            filtered_data.append({
                "DateTime": dt,
                "MODIS_AOD": np.mean(box_aod)
            })

    return pd.DataFrame(filtered_data)

# ============================
# Temporal Collocation
# ============================
def collocate_aeronet_modis(aeronet_df, modis_df, window_minutes=30):
    collocated = []
    aeronet_df['dateTime'] = pd.to_datetime(aeronet_df['dateTime'], format='%m/%d/%Y %I:%M:%S %p')

    for _, row in modis_df.iterrows():
        modis_dt = pd.to_datetime(row["DateTime"])
        modis_aod = row["MODIS_AOD"]

        # Find AERONET measurements within ±30 or ±15  minutes of the MODIS overpass
        mask = (aeronet_df["dateTime"] >= modis_dt - timedelta(minutes=window_minutes)) & \
               (aeronet_df["dateTime"] <= modis_dt + timedelta(minutes=window_minutes))

        matching_aod = aeronet_df.loc[mask, "AOD_550nm"].values
        if len(matching_aod) > 0:
            collocated.append({
                "DateTime": modis_dt,
                "MODIS_AOD": modis_aod,
                "AERONET_AOD": np.mean(matching_aod)
            })

    return pd.DataFrame(collocated)
# print(collocated.head())

# ============================
# Validation Statistics & Plot
# ============================
def plot_validation(collocated_df, title):
    slope, intercept, r_value, p_value, std_err = linregress(
        collocated_df["AERONET_AOD"], collocated_df["MODIS_AOD"]
    )

    mbe = np.mean(collocated_df["MODIS_AOD"] - collocated_df["AERONET_AOD"])
    rmse = np.sqrt(np.mean((collocated_df["MODIS_AOD"] - collocated_df["AERONET_AOD"]) ** 2))
    mae = np.mean(np.abs(collocated_df["MODIS_AOD"] - collocated_df["AERONET_AOD"]))
    r, _ = pearsonr(collocated_df["MODIS_AOD"], collocated_df["AERONET_AOD"])
    num_points = len(collocated_df["AERONET_AOD"])

    x_vals = np.linspace(0, 2, 100)
    ee_bounds = 0.05 + 0.15 * x_vals
    upper = x_vals + ee_bounds
    lower = x_vals - ee_bounds

    collocated_df["EE"] = 0.05 + 0.15 * collocated_df["AERONET_AOD"]
    collocated_df["EE_Upper"] = collocated_df["AERONET_AOD"] + collocated_df["EE"]
    collocated_df["EE_Lower"] = collocated_df["AERONET_AOD"] - collocated_df["EE"]

    within_ee = ((collocated_df["MODIS_AOD"] >= collocated_df["EE_Lower"]) & 
                 (collocated_df["MODIS_AOD"] <= collocated_df["EE_Upper"]))
    percent_within = np.mean(within_ee) * 100

    plt.figure(figsize=(7, 5))
    plt.scatter(collocated_df["AERONET_AOD"], collocated_df["MODIS_AOD"], color='blue', label='Collocated Data')
    plt.plot([0, 2], [0, 2], 'k-', label='1:1 Line')
    plt.plot(x_vals, upper, 'r--', label='EE Upper')
    plt.plot(x_vals, lower, 'r--', label='EE Lower')

    stats = (f'$R^2$ = {r_value**2:.2f}\nSlope = {slope:.2f}\nIntercept = {intercept:.2f}\n'
             f'MBE = {mbe:.2f}\nRMSE = {rmse:.2f}\nMAE = {mae:.2f}\nPearson r = {r:.2f}\n'
             f'% within EE = {percent_within:.1f}%')

    plt.text(0.05, 0.95, f'Regression: y = {slope:.2f}x + {intercept:.2f}',
             transform=plt.gca().transAxes, verticalalignment='top', fontsize=10, bbox=dict(boxstyle="round", facecolor='white'))

    plt.text(0.05, 0.80, stats, transform=plt.gca().transAxes, fontsize=10, verticalalignment='top',
             bbox=dict(boxstyle="round", facecolor='white'))

    plt.text(0.95, 0.95, f'Data Points: {num_points}', transform=plt.gca().transAxes,
             fontsize=10, horizontalalignment='right', verticalalignment='top',
             bbox=dict(boxstyle="round", facecolor='white'))

    plt.xlim(0, 2)
    plt.ylim(0, 2)
    plt.xlabel("AERONET AOD")
    plt.ylabel("MODIS AOD")
    plt.title(title)
    plt.legend()
    plt.grid()
    plt.tight_layout()
    plt.show()


# ============================
# Run the Full Workflow
# ============================
if __name__ == "__main__":
    aeronet_input = r"D:\DATA\COMPLETED\ZAMBIA MONGU_INN\ZAMBIA AERONET\\Zambia aeronet.csv"
    aeronet_output = r"D:\DATA\COMPLETED\ZAMBIA MONGU_INN\ZAMBIA VALIDATION WORK\\aeronet_extracted.csv"
    modis_list = r"D:\DATA\COMPLETED\ZAMBIA MONGU_INN\ZAMBIA MODIS\\fileList.txt"
    modis_dir = r"D:\DATA\COMPLETED\ZAMBIA MONGU_INN\ZAMBIA MODIS"
    modis_output = r"D:\DATA\COMPLETED\ZAMBIA MONGU_INN\ZAMBIA VALIDATION WORK\\modis_extracted.csv"

    process_aeronet_data(aeronet_input, aeronet_output)
    extract_modis_data(modis_list, modis_dir, modis_output)

    bounds = calculate_bounds(aeronet_output)
    modis_df = pd.read_csv(modis_output)
    filtered_modis = spatial_filter_modis_data(modis_df, bounds)

    aeronet_df = pd.read_csv(aeronet_output)
    collocated = collocate_aeronet_modis(aeronet_df, filtered_modis)
    plot_validation(collocated, "ZAMBIA MODIS vs AERONET AOD Validation")


ValueError: Cannot calculate a linear regression if all x values are identical